## 1) Setup

### Importing Dependencies

In [1]:
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten

### Setting up GPU Growth

In [3]:
#to avoid out of memory errors by setting GPU memory consumption growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

### Creating Folder Structures

In [4]:
#Setting up paths
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [5]:
#Making the directories
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

## 2 ) Collecting positives and anchors

### Untaring Labelled Faces in the Wild Dataset

In [6]:
#http://vis-www.cs.umass.edu/lfw/lfw.tgz

In [8]:
#Uncompressing Tar GZ labelled Faces in the Wild Dataset
!tar -xf lfw.tgz

In [11]:
# Moving LFW Images to the following repository data/negative
for directory in os.listdir('lfw'):
    for file in os.listdir(os.path.join('lfw', directory)):
        EX_PATH = os.path.join('lfw', directory, file)
        NEW_PATH = os.path.join(NEG_PATH, file)
        os.replace(EX_PATH, NEW_PATH)

### Collecting Positive and Anchor Classes

In [12]:
#importing uuid library to generate unique image names
import uuid

In [16]:
#Establishing a connection to the webcam
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    #cutting down frame to 250 x 250 px
    frameme = frame[120:120+250, 200:200+250, :]
    
    #collecting anchors
    if cv2.waitKey(1) & 0XFF == ord('a'):
        #creating the unique file path
        imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
        #Writing out positive image
        cv2.imwrite(imgname, frame)
        
    #collecting positives
    if cv2.waitKey(1) & 0XFF == ord('p'):
        #creating the unique file path
        imgname = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        #Writing out positive image
        cv2.imwrite(imgname, frame)
        
    #showing image back to screen
    cv2.imshow('Image Collection', frame)
    
    #Breaking Gracefully
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
# Releasing the webcam
cap.release()
# Closing the image show frame
cv2.destroyAllWindows()